# INDIVIDUAL FOOTPRINTS

In [1]:
import numpy as np
import datetime
import json
import pylab
import pandas as pd
import matplotlib.pyplot as plt

## Preparacion de datos

### Cargando datos

In [2]:

def leer_data():
    outfile='./data/consulta.csv'
    data = pd.read_csv(outfile)
    return data

data_original = leer_data()
data_original.head(3)

,client_id,date,año,mes,dia,hora,merchant_departement,merchant_province,merchant_district,mcc,mccg,client_age,quantity,amount_sol
0,+++g8j9k+5A=,2016-07-07 01:08:05,2016,7,7,1,LIMA,LIMA,San Juan De Luriganc,5655,8,32.0,1,45.0
1,++/oQ9Lb9dI=,2017-05-29 21:05:40,2017,5,29,21,LIMA,LIMA,San Juan De Luriganc,4812,25,49.0,1,104.0
2,++14g8obpj0=,2016-09-08 21:15:55,2016,9,8,21,LIMA,LIMA,San Juan De Luriganc,5912,15,NaN,1,9.0


### Preparacion de datos

In [3]:
data = data_original[['client_id','date','año','mes','dia','hora','quantity','amount_sol']]
data.head(3)   

,client_id,date,año,mes,dia,hora,quantity,amount_sol
0,+++g8j9k+5A=,2016-07-07 01:08:05,2016,7,7,1,1,45.0
1,++/oQ9Lb9dI=,2017-05-29 21:05:40,2017,5,29,21,1,104.0
2,++14g8obpj0=,2016-09-08 21:15:55,2016,9,8,21,1,9.0


## Unidad de TXs temporales (U)

In [4]:

file='U'
raw_data='./data/%s.json' %(file)



### Funciones

In [5]:
# definimos los 4 time_windows que usaremos

def time_window(hora):
    tw = 9999
    if hora >=0:
        tw = 0      # Madrugada
    if hora >=6:
        tw = 1      # Mañana
    if hora >=12:
        tw = 2      # Tarde
    if hora >=18:
        tw = 3      # Noche
    return tw

In [6]:
# Definimos los U 

def procesar_u(user):    
    uid=list(user['client_id'])[0]
    
    # Lista los años en que tiene txs el usuario
    years = set(list(user['año']))
    anni = {year:{} for year in list(years)}
    
    # para cada fila (para cada fecha)
    for dat in  range(0,len(user)):
        año = user.iloc[dat]['año']
        week=datetime.datetime(año,user.iloc[dat]['mes'],user.iloc[dat]['dia']).isocalendar()[1]
        weekday=datetime.datetime(año,user.iloc[dat]['mes'],user.iloc[dat]['dia']).weekday()
        
        # Si la semana no existe en el año
        if not(week in anni[año]):
            anni[año][week]=np.array([0]*7)
            
        anni[año][week][weekday]+=user.iloc[dat]['quantity'] # suma cantidades "importancia por compras"
        #anni[año][week][turn][weekday]+=user.iloc[dat]['amount_sol'] # suma montos "importancia por gastos"
            
    return uid,anni

### Procesando U de cada cliente

In [ ]:
# Extraemos la lista de clientes sin repetir
clientes =  data.groupby('client_id').client_id.count().index
print("",clientes," clientes")
profiles={}
# Para cada cliente
i = 0
for cliente in clientes:
    cliente_i= data[data['client_id'] == cliente]
    ## ejecutamos para cada usuario
    results=procesar_u(cliente_i)
    profiles[results[0]]=results[1]
    if(i%5000==0):
        print(i)
    i+= 1

 Index(['+++g8j9k+5A=', '++/oQ9Lb9dI=', '++14g8obpj0=', '++1XMtcwMec=',
       '++3gxZFOJCM=', '++438ugzEhg=', '++5u+heOZ8o=', '++70ByX0a3Q=',
       '++7i5fi6kBU=', '++834mPfd7g=',
       ...
       'zzvndpR1ntg=', 'zzvt24FsKYk=', 'zzw6JFGkUrQ=', 'zzwY0tP4zKM=',
       'zzxOoziChuk=', 'zzxTJkA/ah0=', 'zzxjK1vzG4E=', 'zzy5LVrTPmc=',
       'zzyhW1+NTLM=', 'zzzBu/yw4wE='],
      dtype='object', name='client_id', length=227663)  clientes
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000


In [ ]:
individual_footprint="%s.individual_footprint" %(raw_data)

#exporting individual footprint   <customer_id;year;week;profile_id;turn;size;t1... tn >
fw=open(individual_footprint,'w')
fw.write('customer_id,year,week,profile_id,size,d0,d1,d2,d3,d4,d5,d6\n')
footprints=0
for uid in profiles:
    profile_id=0
    for year in profiles[uid]:
        for week in profiles[uid][year]:
            d=profiles[uid][year][week]
            fw.write(''+str(uid)+','+str(year)+','+str(week)+','+str(profile_id)+','
                       +str(sum(d))+','+str(d[0])+','+str(d[1])+','+str(d[2])+','+str(d[3])
                       +','+str(d[4])+','+str(d[5])+','+str(d[6])+'\n')

            profile_id = profile_id + 1
    footprints+=profile_id
    fw.flush()
fw.close()
print ("number of footprint: "+str(footprints))